# 压缩文件

> 整理`zipfile`的使用

- [基本操作](#基本操作)
- [读取压缩文件](#读取压缩文件)
- [写入压缩文件](#写入压缩文件)
- [Python ZIP Archives](#Python ZIP Archives)
- [命令行操作](#命令行操作)

参考:
1. https://docs.python.org/3/library/zipfile.html
2. https://pymotw.com/3/zipfile/index.html


## 基本操作

In [27]:
# 判断是否是zip文件
import os
import zipfile

print("判断zip文件,从结果来看这个函数不准确:")
for filename in os.listdir('zipfile'):
      print('{:>15}  {}'.format(
        filename, zipfile.is_zipfile(filename)))

print("查看zip里的文件列表:")        
with zipfile.ZipFile('zipfile/zipfile.zip', 'r') as zf:
    print(zf.namelist())

判断zip文件,从结果来看这个函数不准确:
            dir  False
     readme.txt  False
      resume.md  False
    zipfile.zip  False
查看zip里的文件列表:
['zipfile/', 'zipfile/dir/', 'zipfile/dir/dir_file.txt', 'zipfile/dir/subdir/', 'zipfile/dir/subdir/subdir_file.txt', 'zipfile/readme.txt', 'zipfile/resume.md']


## 读取压缩文件

#### 读取文件

In [28]:
with zipfile.ZipFile('zipfile/zipfile.zip', 'r') as zf:
    for filename in zf.namelist():
        print('{:<30}  :{}'.format(filename,zf.read(filename)))

zipfile/                        :b''
zipfile/dir/                    :b''
zipfile/dir/dir_file.txt        :b'dir_file'
zipfile/dir/subdir/             :b''
zipfile/dir/subdir/subdir_file.txt  :b'subdir_file'
zipfile/readme.txt              :b'my name is hzz, my qq is 510908220'
zipfile/resume.md               :b'# resume\n\n## work\n'


#### 解压单个文件
 `ZipFile.extract(member, path=None, pwd=None)`

- `member`: 文件在压缩包里的名字
- `path`: 目标目录
- `pwd`: 解密密码

In [41]:
from print_util import print_dir
with zipfile.ZipFile('zipfile/zipfile.zip', 'r') as zf:
    zf.extract('zipfile/dir/dir_file.txt', r'zipfile/zip_out')

# 从结果可以看到解压出来的文件保留在zip里的结构
print_dir('zipfile/zip_out')


\zipfile\dir\dir_file.txt


#### 解压全部文件
`ZipFile.extractall(path=None, members=None, pwd=None)`
- `path`: 目标目录
- `members`: `namelist()`的子集
- `pwd`: 解密密码


In [43]:
from print_util import print_dir
with zipfile.ZipFile('zipfile/zipfile.zip', 'r') as zf:
    zf.extractall( 'zipfile/zip_out_all')

print_dir('zipfile/zip_out_all')

\zipfile\readme.txt
\zipfile\resume.md
\zipfile\dir\dir_file.txt
\zipfile\dir\subdir\subdir_file.txt


## 写入压缩文件

`zipfile.ZipFile(file, mode='r', compression=ZIP_STORED, allowZip64=True)`

- `file`: 文件路径或者
- `mode` : 有`r`、`w`、`a`、`x`,很文件差不多. 
- `compression`: 压缩方法
- `allowZip64`: 为`True`时当文件大于`4G`时使用`ZIP64`扩展.

In [64]:
import zipfile

with zipfile.ZipFile('zipfile/write_zip/write.zip', mode='w') as zf:
    zf.write('zipfile/readme.txt') # 写入文件
    zf.writestr('from_string.txt', "msg") # 写入字符串
print_dir('zipfile/write_zip')

\write.zip


In [65]:
import zipfile

with zipfile.ZipFile('zipfile/write_zip/write.zip', mode='a') as zf:
    zf.write('zipfile/resume.md') # 追加文件
print_dir('zipfile/write_zip')

\write.zip


## Python ZIP Archives
>  python可以从zip里导入模块

`
zipfile.PyZipFile(file, mode='r', compression=ZIP_STORED, allowZip64=True, optimize=-1)
`


#### 创建python zip包

In [66]:
with zipfile.PyZipFile('zipfile/pyzipfile.zip', mode='w') as zf:
    zf.debug = 3
    print('Adding python files')
    zf.writepy('zipfile/pyzipfile')
for name in zf.namelist():
    print(name)

Adding python files
Adding package in zipfile/pyzipfile as pyzipfile
Compiling zipfile/pyzipfile\__init__.py
Adding pyzipfile/__init__.pyc
Compiling zipfile/pyzipfile\test_module.py
Adding pyzipfile/test_module.pyc
pyzipfile/__init__.pyc
pyzipfile/test_module.pyc


#### 使用python zip包

In [68]:
import sys
sys.path.insert(0, 'zipfile/pyzipfile.zip')

from pyzipfile import test_module
test_module.show_time()


2017-03-24 09:57:05.485444


## 命令行操作
> `zipfile`提供了一些简单的命令行接口去和zip交互

- 压缩文件:`python -m zipfile -c monty.zip spam.txt eggs.txt`
- 压缩目录: `python -m zipfile -c monty.zip life-of-brian_1979/`
- 解压到某个目录: `python -m zipfile -e monty.zip target-dir/`
- 查看压缩包内容: `python -m zipfile -l monty.zip`
